In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# needed for notebook to find other directories
module_path = os.path.abspath(os.path.join('../waymo-od'))
if module_path not in sys.path:
    sys.path.append(module_path)
# needed for notebook to find other directories

In [ ]:
import os
import tensorflow.compat.v1 as tf
import math
import numpy as np
import itertools

tf.enable_eager_execution()

from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset
from waymo_open_dataset.utils import box_utils

#from waymo_open_dataset.camera.ops import py_camera_model_ops

from scipy.spatial.transform import Rotation as R

import matplotlib.pyplot as plt
import open3d as o3d

In [ ]:
FILENAME = '/home/tristram/data/waymo/training-segment-10876852935525353526_1640_000_1660_000_with_camera_labels.tfrecord'
dataset = tf.data.TFRecordDataset(FILENAME, compression_type='')
frames = []
for data in dataset:
    frame = open_dataset.Frame()
    frame.ParseFromString(bytearray(data.numpy()))
    frames.append(frame)
    

In [ ]:
frame = frames[0]
print(frame.laser_labels)

In [ ]:
id1 = frame.laser_labels[0].id
id2 = frames[4].laser_labels[0].id
print(id1, id2)
#print(frame.camera_labels[1].labels[1])
#print(frames[1].camera_labels[0].labels[0])
cx0 = frames[0].laser_labels[0].box.center_x
cx1 = frames[4].laser_labels[0].box.center_x
cy0 = frames[0].laser_labels[0].box.center_y
cy1 = frames[4].laser_labels[0].box.center_y
cz0 = frames[0].laser_labels[0].box.center_z
cz1 = frames[4].laser_labels[0].box.center_z

coors0 = np.array([cx0, cy0, cz0])
coors1 = np.array([cx1, cy1, cz1])

t0 = np.asarray(frames[0].pose.transform).reshape(4,4)[:3,3]
t4 = np.asarray(frames[4].pose.transform).reshape(4,4)[:3,3]

coors1 = coors1 + (t0 - t4)
print(coors1)
print(coors0)
print(coors1 - coors0)
print(coors1 + (coors0 - coors1))

In [ ]:
ids = []
cams = ['_FRONT', '_SIDE']
for i, frame in enumerate(frames):
    if i < 5:
        for i, elem in enumerate(frame.projected_lidar_labels):
                for labels in elem.labels:
                    if labels.type == 1:
                        if np.abs(labels.metadata.speed_x * labels.metadata.speed_y) > 10.0:
                            for cam in cams:
                                if cam in labels.id:
                                    if labels.id.split(cam)[0] not in ids:
                                        ids.append(labels.id.split(cam)[0])
                    """elif labels.type == 2:
                        if np.abs(labels.metadata.speed_x * labels.metadata.speed_y) > 0.1:
                            for cam in cams:
                                if cam in labels.id:
                                    if labels.id.split(cam)[0] not in ids:
                                        ids.append(labels.id.split(cam)[0])"""
                                    
print(np.array(ids))

In [ ]:
# Assign each object a number to identify it by in the mask later

id_dict = {}
for i in range(len(ids)):
    id_dict[ids[i]] = i + 1
    
print(id_dict)

In [ ]:
canonical = {}
delta = {}
for i, frame in enumerate(frames):
    if 0 < i < 10:
        cr_pose = np.asarray(frame.pose.transform).reshape(4,4)[:3,3]
        mvmt = cn_pose - cr_pose
        for key in id_dict:
            for elem in frame.laser_labels:
                if elem.id == key:
                    # add offset from vehicle movement to bbox centers
                    coors = np.array([elem.box.center_x, elem.box.center_y, elem.box.center_z])
                    coors += mvmt
                    # re-order coordinates from laser to nerf coordinate system
                    coors = np.array([-coors[1], coors[2], -coors[0]])
                    # this delta has to be added to move to canonical space (not subtracted)
                    delta[str(i)+'_'+str(id_dict[key])] = canonical[key] - coors

    elif i == 0:
        cn_pose = np.asarray(frame.pose.transform).reshape(4,4)[:3,3]
        for key in id_dict:
            for elem in frame.laser_labels:
                if elem.id == key:
                    # re-order coordinates from laser to nerf coordinate system
                    canonical[key] = np.array([-elem.box.center_y, elem.box.center_z, -elem.box.center_x])
print(canonical)
print(delta)
#np.save('/home/tristram/data/waymo/seg1_5_center/delta.npy', delta)

In [ ]:
def aa2matrix_old(angles):
    """This implements going from axis-angle to a rotation matrix representation
        Args:
            angles: rotation around each axis [x, y, z] in radians
        Returns:
            R: 3x3 rotation matrix
    """
    x, y, z = angles[:3]
    skew_r = np.array([[0., -z, y],
                        [z, 0., -x],
                        [-y, x, 0.]])
    angles_norm = np.linalg.norm(angles) + 1e-12
    R = np.eye(3) + (np.sin(angles_norm) / angles_norm) * skew_r + \
        ((1 - np.cos(angles_norm)) / angles_norm**2) * np.matmul(skew_r, skew_r)
    return R

In [ ]:
dict_3d = {}
data_dir = '/home/tristram/data/waymo/seg2_5/'

opengl2waymo = np.array([[0, 0, -1, 0],
                        [-1, 0, 0, 0],
                        [0, 1, 0, 0],
                        [0, 0, 0, 1]])

trafo2 = np.array([[-1, 0, 0, 0],
                        [0, 0, 1, 0],
                        [0, 1, 0, 0],
                        [0, 0, 0, 1]])

for i, frame in enumerate(frames):
    if 0 < i < 5:
        v2w = np.asarray(frame.pose.transform).reshape(4,4)

        for key in id_dict:
            for elem in frame.laser_labels:
                if elem.id == key:
                    can_pose = np.eye(4)
                    can_pose[:3,3] = np.array([elem.box.center_x, elem.box.center_y, elem.box.center_z])
                    axis_angle = np.array([0.0, 0.0, elem.box.heading])
                    rot = aa2matrix_old(axis_angle)
                    can_pose[:3, :3] = rot
                    can_pose = np.matmul(v2w, can_pose)
                    can_pose = np.matmul(can_pose, opengl2waymo)
                    can_pose = np.matmul(trafo2, can_pose)
                    car = id_dict[key]
                    extent = np.array([elem.box.width, elem.box.height, elem.box.length])
                    #extent = np.array([elem.box.length, elem.box.width, elem.box.height])
                    dict_3d[str(i+1)+'_'+str(car)+'_center'] = can_pose
                    dict_3d[str(i+1)+'_'+str(car)+'_ext'] = extent
    elif i == 0:
        v2w = np.asarray(frame.pose.transform).reshape(4,4)
        for key in id_dict:
            for elem in frame.laser_labels:
                if elem.id == key:
                    can_pose = np.eye(4)
                    can_pose[:3,3] = np.array([elem.box.center_x, elem.box.center_y, elem.box.center_z])
                    axis_angle = np.array([0.0, 0.0, elem.box.heading])
                    rot = aa2matrix_old(axis_angle)
                    can_pose[:3, :3] = rot
                    can_pose = np.matmul(v2w, can_pose)
                    can_pose = np.matmul(can_pose, opengl2waymo)
                    can_pose = np.matmul(trafo2, can_pose)
                    car = id_dict[key]
                    extent = np.array([elem.box.width, elem.box.height, elem.box.length])
                    #extent = np.array([elem.box.length, elem.box.width, elem.box.height])
                    dict_3d[str(i+1)+'_'+str(car)+'_center'] = can_pose
                    dict_3d[str(i+1)+'_'+str(car)+'_ext'] = extent
                    
np.save(data_dir+'3D_boxes.npy', dict_3d)
print(dict_3d)

In [ ]:

def project_point(point, camera_calibration):
      # vehicle frame to camera sensor frame.
    extrinsic = np.array(camera_calibration.extrinsic.transform).reshape(4, 4)
    vehicle_to_sensor = np.linalg.inv(extrinsic)
    point1 = np.concatenate([point, [1,]])
    point_camera_frame = tf.einsum('ij,j->i', extrinsic, tf.constant(point1, dtype=tf.float32))
    u_d = - point_camera_frame[1] / point_camera_frame[0]
    v_d = - point_camera_frame[2] / point_camera_frame[0]

    # add distortion model here if you'd like.
    f_u = camera_calibration.intrinsic[0];
    f_v = camera_calibration.intrinsic[1];
    c_u = camera_calibration.intrinsic[2];
    c_v = camera_calibration.intrinsic[3];
    u_d = u_d * f_u + c_u;
    v_d = v_d * f_v + c_v;

    return [u_d.numpy(), v_d.numpy()]

In [ ]:
import cv2
points = []
frame = frames[0]
center = dict_3d['1_1_center'][:3, 3]
heading = R.from_matrix(dict_3d['1_1_center'][:3, :3])
heading = heading.as_rotvec()[-1]
ext = dict_3d['1_1_ext']
l, w, h = ext / 2
points.append(center + np.array([l, w, h]))
points.append(center + np.array([-l, w, h]))
points.append(center + np.array([l, -w, h]))
points.append(center + np.array([l, w, -h]))
points.append(center + np.array([l, -w, -h]))
points.append(center + np.array([-l, -w, h]))
points.append(center + np.array([-l, w, -h]))
points.append(center + np.array([-l, -w, -h]))
points = np.array(points)

#py_camera_model_ops.world_to_image()

box = tf.convert_to_tensor(np.concatenate([center, ext, [heading,]]))
box = tf.reshape(box, (1, 7))

p = box_utils.get_upright_3d_box_corners(box)
print(p)
img_p = []
for i in range(8):
    img_p.append(project_point(np.array(p[0, i]), frame.context.camera_calibrations[0]))

img_p = np.array(img_p)
print(img_p)
img = np.array(tf.image.decode_jpeg(frame.images[1].image))

K0 = np.array(frames[100].context.camera_calibrations[3].intrinsic)
K1 = np.array(frame.context.camera_calibrations[1].intrinsic)
K2 = np.array(frame.context.camera_calibrations[2].intrinsic)
K3 = np.array(frame.context.camera_calibrations[3].intrinsic)
K4 = np.array(frame.context.camera_calibrations[4].intrinsic)
print(K0[:4])
print(K1[:4])
print(K2[:4])
print(K3[:4])
print(K4[:4])
print(frame.context.camera_calibrations)

In [ ]:
poses = np.load('/home/tristram/data/waymo/seg1_5_center/poses_bounds.npy')
poses = poses[:, :-2].reshape(-1, 3, 5)
poses = poses[:, :, :4]
poses = np.concatenate([poses, np.broadcast_to([0, 0, 0, 1], (poses.shape[0], 1, 4))], axis=1)

arr = []
for keys in dict_3d:
    if 'center' in keys:
        arr.append(dict_3d[keys])
arr = np.array(arr)
#print(arr)
poses = np.concatenate([poses, arr], axis=0)
np.save('/home/tristram/data/waymo/boxpose.npy', poses)

In [ ]:
data_dir = '/home/tristram/data/waymo/seg2_5/'
order = [1, 2, 4, 3, 5]
masks = []
for i, frame in enumerate(frames):
    if i < 5:
        for j, elem in enumerate(frame.projected_lidar_labels):
            if elem.name < 4:
                mask = np.zeros((320,480)).astype(np.int8)
                for labels in elem.labels:
                    if labels.type == 1:

                        if np.abs(labels.metadata.speed_x * labels.metadata.speed_y) > 10.0:
                            left_x = np.ceil((labels.box.center_x / 4.0) - 0.5 * (labels.box.length / 4.0))
                            right_x = np.ceil((labels.box.center_x / 4.0) + 0.5 * (labels.box.length / 4.0))
                            left_y = np.ceil((labels.box.center_y / 4.0) - 0.5 * (labels.box.width / 4.0))
                            right_y = np.ceil((labels.box.center_y / 4.0) + 0.5 * (labels.box.width / 4.0))
                            obj_num = id_dict[labels.id.split('_FRONT')[0]]
                            mask[int(left_y):int(right_y), int(left_x):int(right_x)] = int(obj_num)
                    """elif labels.type == 2:

                        if np.abs(labels.metadata.speed_x * labels.metadata.speed_y) > 0.1:
                            left_x = np.ceil((labels.box.center_x / 4.0) - 0.5 * (labels.box.length / 4.0))
                            right_x = np.ceil((labels.box.center_x / 4.0) + 0.5 * (labels.box.length / 4.0))
                            left_y = np.ceil((labels.box.center_y / 4.0) - 0.5 * (labels.box.width / 4.0))
                            right_y = np.ceil((labels.box.center_y / 4.0) + 0.5 * (labels.box.width / 4.0))
                            obj_num = id_dict[labels.id.split('_FRONT')[0]]
                            mask[int(left_y):int(right_y), int(left_x):int(right_x)] = obj_num"""
            else:
                mask = np.zeros((221,480)).astype(np.int8)
                for labels in elem.labels:
                    if labels.type == 1:

                        if np.abs(labels.metadata.speed_x * labels.metadata.speed_y) > 10.0:
                            left_x = np.ceil((labels.box.center_x / 4.0) - 0.5 * (labels.box.length / 4.0))
                            right_x = np.ceil((labels.box.center_x / 4.0) + 0.5 * (labels.box.length / 4.0))
                            left_y = np.ceil((labels.box.center_y / 4.0) - 0.5 * (labels.box.width / 4.0))
                            right_y = np.ceil((labels.box.center_y / 4.0) + 0.5 * (labels.box.width / 4.0))
                            obj_num = id_dict[labels.id.split('_SIDE')[0]]
                            mask[int(left_y):int(right_y), int(left_x):int(right_x)] = int(obj_num)
                    """elif labels.type == 2:

                        if np.abs(labels.metadata.speed_x * labels.metadata.speed_y) > 0.1:
                            left_x = np.ceil((labels.box.center_x / 4.0) - 0.5 * (labels.box.length / 4.0))
                            right_x = np.ceil((labels.box.center_x / 4.0) + 0.5 * (labels.box.length / 4.0))
                            left_y = np.ceil((labels.box.center_y / 4.0) - 0.5 * (labels.box.width / 4.0))
                            right_y = np.ceil((labels.box.center_y / 4.0) + 0.5 * (labels.box.width / 4.0))
                            obj_num = id_dict[labels.id.split('_SIDE')[0]]
                            mask[int(left_y):int(right_y), int(left_x):int(right_x)] = obj_num"""
            masks.append(mask)

In [ ]:
for i in range(int(len(masks) / 5)):
    temp = masks[5*i+2]
    masks[5*i+2] = masks[5*i+3]
    masks[5*i+3] = temp

In [ ]:
all_masks = np.array(masks)

print(len(all_masks))
np.savez(data_dir+"2D_boxes.npz", all_masks)

## Create Delta Image
Assign a delta to every pixel according to its timestep and object. 


Uses delta and object masks calculated in the cells above

In [ ]:
# create delta offset from canonical pose for every timestep and pixel

timestep = 0
delta_imgs = []
for i, m in enumerate(masks):
    if i > 0 and i % 5 == 0:
        timestep +=1
    dx_img = np.zeros((m.shape[0],m.shape[1], 3))
    for keys in delta:
        step, obj = keys.split('_')
        step = int(step)
        obj = int(obj)
        if step == timestep:
            sel = (m == obj)
            dx_img[sel] = delta[keys]
    delta_imgs.append(dx_img)

In [ ]:
for d in delta_imgs:
    dr = d.reshape(-1,3)
    for i in range(dr.shape[0]):
        if dr[i].any() > 0:
            print(dr[i])
            break

In [ ]:
delta_imgs = np.array(delta_imgs)
print(len(delta_imgs))
np.savez(data_dir+'delta_masks.npz', delta_imgs)

In [ ]:
all_masks = np.load('/home/tristram/data/waymo/seg1_5_center/'+'dynamic_masks_test.npz', allow_pickle=True)['arr_0']

for m in all_masks:
    plt.figure()
    plt.imshow(m)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def show_camera_image(camera_image, camera_labels, layout, cmap=None):
  """Show a camera image and the given camera labels."""

  ax = plt.subplot(*layout)

  # Draw the camera labels.
  for camera_label in camera_labels:
    # Ignore camera labels that do not correspond to this camera.
    if camera_label.name != camera_image.name:
      continue

    # Iterate over the individual labels.
    for label in camera_label.labels:
        if label.type == 1 or label.type == 2:
            if np.abs(label.metadata.speed_x * label.metadata.speed_y) > 10.0:
                  # Draw the object bounding box.
                  ax.add_patch(patches.Rectangle(
                    xy=(label.box.center_x - 0.5 * label.box.length,
                        label.box.center_y - 0.5 * label.box.width),
                    width=label.box.length,
                    height=label.box.width,
                    linewidth=1,
                    fill=True,
                    edgecolor='red',
                    facecolor='none'))

  # Show the camera image.
  plt.imshow(tf.image.decode_jpeg(camera_image.image), cmap=cmap)
  plt.title(open_dataset.CameraName.Name.Name(camera_image.name))
  plt.grid(False)
  plt.axis('on')

plt.figure(figsize=(15, 10))
for index, image in enumerate(frames[0].images):
  show_camera_image(image, frames[0].projected_lidar_labels, [3, 3, index+1])